Q-8. Quora question pair similarity, you need to find the Similarity between two
questions by mapping the words in the questions using TF-IDF, and using a supervised
Algorithm you need to find the similarity between the questions.
Dataset This is the Dataset You can use this dataset for this question.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={"figure.figsize":(30,15)})
pd.pandas.set_option("display.max_columns",None)

In [2]:
df = pd.read_csv('C:/Users/himanshu/ml project code/jupyter books/ml_assignment_code/train.csv')
df_copy = df.copy()

In [24]:
df_copy.shape

(404287, 5)

In [3]:
df_copy

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [4]:
df_copy['question1'].unique()

array(['What is the step by step guide to invest in share market in india?',
       'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
       'How can I increase the speed of my internet connection while using a VPN?',
       ..., 'What is one coin?',
       'What is the approx annual cost of living while studying in UIC Chicago, for an Indian student?',
       'What is like to have sex with cousin?'], dtype=object)

In [36]:
df_copy.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [5]:
df_copy.dropna(inplace=True)

In [11]:
import re
import string

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Apply any additional preprocessing steps as needed
    
    return text


In [7]:
df_copy.duplicated().sum()

0

In [8]:
numerical_feature = [feature for feature in df_copy.columns if df_copy[feature].dtypes != 'O']
discrete_feature=[feature for feature in numerical_feature if len(df_copy[feature].unique())<25]
continuous_feature = [feature for feature in numerical_feature if feature not in discrete_feature]
categorical_feature = [feature for feature in df_copy.columns if feature not in numerical_feature]

In [9]:
categorical_feature

['question1', 'question2']

In [10]:
for i in categorical_feature:
    print(i)
    print(df_copy[i].unique())
    print(len(df_copy[i].unique()))
    print('*'*20)

question1
['What is the step by step guide to invest in share market in india?'
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?'
 'How can I increase the speed of my internet connection while using a VPN?'
 ... 'What is one coin?'
 'What is the approx annual cost of living while studying in UIC Chicago, for an Indian student?'
 'What is like to have sex with cousin?']
290455
********************
question2
['What is the step by step guide to invest in share market?'
 'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?'
 'How can Internet speed be increased by hacking through DNS?' ...
 "What's this coin?"
 'I am having little hairfall problem but I want to use hair styling product. Which one should I prefer out of gel, wax and clay?'
 'What is it like to have sex with your cousin?']
299173
********************


In [12]:
df_copy['question1'] = df_copy['question1'].apply(preprocess_text)

In [13]:
df_copy['question2'] = df_copy['question2'].apply(preprocess_text)

In [15]:
df_copy.drop('id',axis=1,inplace=True)

In [17]:
df_copy

,qid1,qid2,question1,question2,is_duplicate
0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0
1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...,0
2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...,0
3,7,8,why am i mentally very lonely how can i solve it,find the remainder when mathmath is divided by,0
4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water,0
...,...,...,...,...,...
404285,433578,379845,how many keywords are there in the racket prog...,how many keywords are there in perl programmin...,0
404286,18840,155606,do you believe there is life after death,is it true that there is life after death,1
404287,537928,537929,what is one coin,whats this coin,0
404288,537930,537931,what is the approx annual cost of living while...,i am having little hairfall problem but i want...,0


In [18]:
X = df_copy.drop('is_duplicate',axis=1)
y = df_copy['is_duplicate']

In [20]:
X

,qid1,qid2,question1,question2
0,1,2,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,3,4,what is the story of kohinoor kohinoor diamond,what would happen if the indian government sto...
2,5,6,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,7,8,why am i mentally very lonely how can i solve it,find the remainder when mathmath is divided by
4,9,10,which one dissolve in water quikly sugar salt ...,which fish would survive in salt water
...,...,...,...,...
404285,433578,379845,how many keywords are there in the racket prog...,how many keywords are there in perl programmin...
404286,18840,155606,do you believe there is life after death,is it true that there is life after death
404287,537928,537929,what is one coin,whats this coin
404288,537930,537931,what is the approx annual cost of living while...,i am having little hairfall problem but i want...


In [21]:
y

0         0
1         0
2         0
3         0
4         0
         ..
404285    0
404286    1
404287    0
404288    0
404289    0
Name: is_duplicate, Length: 404287, dtype: int64

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.40,random_state=40)

In [23]:
X_train.shape

(242572, 4)

In [25]:
X_test.shape

(161715, 4)

In [26]:
y_train.shape

(242572,)

In [27]:
y_test.shape

(161715,)

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [30]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['question1'] + ' ' + X_train['question2'])
X_test_tfidf = vectorizer.transform(X_test['question1'] + ' ' + X_test['question2'])


In [31]:
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7523853693225737


c:\Users\himanshu\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



TfidfVectorizer() is a class in the scikit-learn library that is used to convert a collection of raw documents into a matrix of TF-IDF features. TF-IDF stands for Term Frequency-Inverse Document Frequency, and it is a commonly used technique in natural language processing to represent the importance of a term in a document relative to a corpus.

In [33]:
model = RandomForestClassifier()
model.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8052376093745169
